In [20]:
import numpy as np
import scipy.sparse
import math
from scipy.sparse import  find

def BinEm(X):
    '''Catogrical to binray conversion'''
    I = find(X)
    X_new = np.zeros(X.shape, dtype = np.uint32)
    for i in range(len(I[0])):
        if np.random.rand() < 0.5:
             X_new[I[0][i], I[1][i]] = 1
        else:
             X_new[I[0][i], I[1][i]] = 0
    return scipy.sparse.csr_matrix(X_new)

def BinSketch(X, k):
    '''Sketch generation using BinSketch: This method takes a binary matrix X of size nxd, and generates its sketch matrix of size nxk, where k<<d. 
        Parameters:
           X  is a sparse array (NumPy array or can be scipy sparse matrix) of shape (n, d),
              where n is the number of samples and d is feature dimension
           k is the  reduced dimension value (must be a positive integer)
            
        Returns:
          Sketch matrix of X of size nxk-- scipy sparse csr_matrix of shape (n, k)
    '''


    d = X.shape[1]
    random_map = np.array([], int)
    M = np.arange(k)
    for i in range(d//k):
        np.random.shuffle(M)
        random_map = np.concatenate((random_map, M))
    np.random.shuffle(M)
    random_map = np.concatenate((random_map, M[0:d%k]))
    np.random.shuffle(random_map)

    new_X = np.zeros((X.shape[0], k))
    for i in range(k):
        index = np.where(random_map == i)[0]
        new_X[:,i] = np.max(X[:,index].A, axis = 1)
        
    return scipy.sparse.csr_matrix(new_X)

def Cabin(X,k):
    X_Bin = BinEm(X)
    X_Bin_Sketch = BinSketch(X_Bin, k)
    return X_Bin_Sketch

def BinSketch_Sparsity_Estimate(a):
    '''This method takes a low-dimensional (sketch) binary vectors a obtained from the BinSketch method and estimate the sparsity for full-dimesnion version.
        Parameters:
            Low-dimensional (sketch) binary vector a
        Returns:
            An estimate of the sparsity for full-dimensional version of a
    '''
    N = a.shape[1]
    spar_a = a.nnz   #nnz is scipy sparse matrix method to find number of nonzero entries
    
    if spar_a/N < 1:
        spar_est_a = round(math.log(1-(spar_a/N))/math.log(1- 1/N))
    else:
        spar_est_a = spar_a
    return spar_est_a

def BinSketch_IP_Estimate(a, b):
    ''' This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the Inner product between their full-dimensional version.
        Parameters:
            Low-dimensional (sketch) binary vectors a, b  obtained from BinSketch 

        Returns:
            An estimate of the Inner product between full-dimensional vectors corresponding to a and b
    '''
    N = a.shape[1]
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
  
    IP = int(a.dot(b.T)[0,0])
    val = (1 - 1/N)**(spar_est_a) + (1 - 1/N)**(spar_est_b) + (IP/N) - 1
    if  val > 0:
        Bin_IP_Est  = round(spar_est_a + spar_est_b - (math.log(val) / math.log(1 - 1/N)))
        if Bin_IP_Est < 0:
            Bin_IP_Est = IP
    else:
        Bin_IP_Est = IP
    return Bin_IP_Est
   
def Cham(a, b):
    '''This method takes two low-dimensional (sketch) binary vectors a and b obtained from the BinSketch method and estimate the hamming distance between their full-dimensional version.
        Parameters:
            binary vectors a, b 
        Returns:
            An estimate of the Hamming distance between  full-dimensional vectors corresponding to a and b

    '''
    spar_est_a = BinSketch_Sparsity_Estimate(a)
    spar_est_b = BinSketch_Sparsity_Estimate(b)
    Bin_IP_Est = BinSketch_IP_Estimate(a, b)
    
    Cham_Ham_Est = 2*(spar_est_a + spar_est_b - 2*Bin_IP_Est)
    
    return Cham_Ham_Est

def Hamming_distance(a,b):
    'Funtion to calculate hamming distance between array a and b'
    ham = 0
    for i in range(a.shape[1]):
        if a[:,i] != b[:,i]:
            ham += 1
    return ham


In [22]:
#Example
X = scipy.sparse.load_npz('Sample.npz')
print('Shape of actual matrix:', X.shape)

new_X = Cabin(X, 1000)
print('Shape of Cabin Sketch: ', new_X.shape)

Shape of actual matrix: (100, 102660)
Shape of Cabin Sketch:  (100, 1000)


In [23]:
#hamming estimate
a = X[0,:]
b = X[1,:]
#hamming distance between a and b taken from actual matrix
print('Hamming distance between a and b is :', Hamming_distance(a,b))

a_new = new_X[0,:]
b_new = new_X[1,:]
# a_new and b_new corresponds to compressed sketch of a and b
print('Hamming estimate of a and b  using Cabin_cham sketch is :', Cham(a_new, b_new))

Hamming distance between a and b is : 225
Hamming estimate of a and b  using Cabin_cham sketch is : 224
